@author: Boda Ye
This python file used to label conn log file by using honeypot logfile.

        def ip_list:        create honeypot ip_time dictionary
                            like, { ip1: {time1,time2}, ip2:{time5, time 6, time7}}



    class label: utilize the ip_time dicitonary to label the conn log file



        def subset_conn:    This function is used to subset the conn log file, pick out rows with 'malicious ip'
                            (ip caught in honeypot)



        def preprocess_ip:  This function is used to convert the ip_time dicitonary to ip_start_end dictionary 
                            The unit for window is sec
                            
                            example:
                            ip1: {10,15,100} window=100, ts_threshold=13
                            => ip1:{10,100}
                            => ip1:{10-110, 100-200} 
                            => ip1:{10-200}
                           
                            

        def mal_uid:        return a list of malicious uid
    

In [1]:
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime

In [2]:

#create honeypot ip_time dictionary
#like, { ip1: {time1,time2}, ip2:{time5, time 6, time7}}
#input honeypotfile should be csv
def ip_list(honeypot):
    start_time = time.time()
    hp=pd.read_csv(honeypot)
    #read in honeypot file,and transfer timestamp to unix format
    hp.timestamp = [time.mktime(datetime.strptime(x[:18], '%Y-%m-%d %H:%M:%S').timetuple())\
            for x in hp.timestamp]

    #create honeypot ip_time dictionary
    #like, { ip1: {time1,time2}, ip2:{time5, time 6, time7}}
    df=hp.iloc[:,[1,4]]
    ip_time={}
    for index,row in df.iterrows():
        ts,sip=int(row[0]),row[1].strip()
        if sip=='0':continue
        if sip not in ip_time.keys():
            ip_time[sip]={ts}
        else:
            ip_time[sip].add(ts)

    print("--- %s seconds ---" % (time.time() - start_time))
    #return a dictionary
    return ip_time

In [3]:
#create honeypot ip_time dictionary
#like, { ip1: {time1,time2}, ip2:{time5, time 6, time7}}
ip_time=ip_list('/scratch/by8jj/honeypot2_1.csv')

--- 307.89243626594543 seconds ---


In [35]:
class label(object):
    
    
    def __init__(self,conn):
        self.conn=conn
        
    
    '''
    #This function is used to subset the conn log file
    def sub_conn(self,ip_time):
        
        start_time = time.time()
        malicious_ip=[x for x in ip_time.keys()]
        
        #read in the file
        f = open(self.conn, 'r')
        
        #initialize out
        out=[]
        
        line=f.readline().strip()
        
        counter=1
        #subset the file
        while line:
            counter+=1
            lsplit = line.split('\t')
            try:
                ts, uid, sip = lsplit[0][:10],lsplit[1], lsplit[2]
                if sip in malicious_ip:out.append([ts,uid,sip])
            except:
                if 'open' in line: print(line)
            line = f.readline().strip() 
        
        print('original size is:%s  subset size is %s'%(counter, len(out)))
        f.close()
        print('sub_conn: -------%s seconds ------'%(time.time()-start_time))
        return out
        '''
        
    
    #a mini function to check whether there's overlap between 2 duration
    def overlap(self,a,b):
        if a[1]<b[0]:return True
        return False
    
    
    #This function is used to convert the ip_time dicitonary to ip_start_end dictionary 
    #The unit for window is sec
    def preprocess_ip(self,ip_time,ts_threshold=13,window=43200):
        
        start_time=time.time()
        #To simplify problem, I assume logs with same ip in 13 secs are from exactly the same malicious hacker
        #choose 13 here is because of mean duration for connection attempts is 13 secs
        
        out={}
        for key,value in ip_time.items():

            #combine connection happened within 13 secs to reduce size of time_stamp file
            ts=[int(x) for x in value]
            temp=[ts[0]]
            for j in range(len(ts)-1):
                if ts[j+1]-ts[j]>ts_threshold: temp.append(ts[j+1])           


            #create the start_end for each item in temp
            temp=[(x,x+window) for x in temp]


            #merge start_end
            merged_start_end=[temp[0]]
            for j in range(1,len(temp)):
                current_start_end=merged_start_end.pop()
                if self.overlap(current_start_end,temp[j]):
                    merged_start_end.append((current_start_end[0],temp[j][1]))
                else:
                    merged_start_end.append(current_start_end)
                    merged_start_end.append(temp[j])
            out[key]=merged_start_end
    
        #return a dictionary 
        print('preprocess ip:--------%s seconds ----------'%(time.time()-start_time))
        return out
        
        
    #label the file
    def label(self,ip_time,outfile='labeled.log',l_ts_threshold=13,l_window=43200):
        
        #intitialize some variable here
        start_time=time.time()
        mali_count=0
        
        
        #run preprocess function
        preprocess_ip_time=self.preprocess_ip(ip_time,ts_threshold=l_ts_threshold,window=l_window)
        
        
        #initialize the output file
        out=open(outfile,'w')
        
        
        #read in the file
        f = open(self.conn, 'r')
        
        line=f.readline().strip()
        
        counter=1
        
        while line:
            
            if '#' in line:
                line=f.readline().strip()
                counter+=1
                continue
                
            lsplit = line.split('\t')     
            ts, uid, ip = lsplit[0][:10],lsplit[1], lsplit[2]

            #check if ip is caught in honeypot file
            if ip not in preprocess_ip_time.keys():
                out.write(line+'\t'+'0'+'\n')
                line=f.readline().strip()
                counter+=1
                continue

            #check if time is within the range on the preprocessed ip_time file
            for j in preprocess_ip_time[ip]:
                if int(ts)>=int(j[0]) and int(ts)<=int(j[1]):
                    mali_count+=1
                    out.write(line+'\t'+'1'+'\n')
                    break

            line=f.readline().strip()
            out.write(line+'\t'+'0'+'\n')

            counter+=1
        
        f.close()
        out.close()
        print('size of original file is:%s, size of malicious ip in conn log file is:%s '%(counter, mali_count))
        print('label:--------%s seconds -------'%(time.time()-start_time))
        return mali_count
        

In [36]:
conn='/scratch/by8jj/conn_log_sample/merged_tcp.log'
p=label(conn)
p.label(ip_time)

preprocess ip:--------0.023392915725708008 seconds ----------
size of original file is:27880564, size of malicious ip in conn log file is:435498 
label:--------53.88567280769348 seconds -------


435498

In [6]:
#implement sensitivity analysis to decide the best time window
def mali_sensitivity(start=10800,end=21600,step=3600):
    res=[]
    for i in range(start,end,step):      
        temp=p.label(ip_time,window=i)
        res.append(temp)
        print(temp)
        print("--- %s seconds ---" %(time.time() - start_time))
    return res

In [37]:
conn='/scratch/by8jj/conn_log_sample/merged_tcp.log'
labeled='/scratch/by8jj/window_function/labeled.log'

In [38]:
#test
def test(file):
    f=open(file,'r')

    for i in range(100):
        line=f.readline().strip()
        print(line)
    

In [39]:
test(labeled)

1534723192.365457	Co7jJo3ZWvcHNVJBp3	3.110.248.189	24472	13.107.5.88	443	2.234384	819	6093	RSTO	ShADadR	17	13	(empty)	1	0
1534723194.600236	ChNGWc2tYAUfXsSAOf	45.227.255.97	40351	3.110.130.108	8549	-	-	-	S0	S	1	0	(empty)	0	0
1534723194.603546	CcFOOY1Ws7QH2xI2Al	45.227.255.97	40340	3.110.95.246	5569	-	-	-	S0	S	1	0	(empty)	0	0
1534723194.604892	CZPyv12tvOYvGbdpMl	45.227.255.97	40340	3.110.3.29	5580	-	-	-	S0	S	1	0	(empty)	0	0
1534723194.607115	CK9nil4OJcbHqHXZk3	185.169.231.209	65535	3.110.152.124	8888	-	-	-	S0	S	1	0	(empty)	0	0
1534723194.607921	CAyQv12zpLXt4qOGLf	5.189.226.178	60000	3.110.122.8	7000	-	-	-	S0	S	1	0	(empty)	0	0
1534723194.612035	CsxACnBrCcVwwFFB8	60.161.202.12	11835	3.110.206.104	23	-	-	-	S0	S	1	0	(empty)	0	0
1534723194.612120	C0lyD023HM6QI6Ng43	80.82.64.116	40195	3.110.237.90	4089	-	-	-	S0	S	1	0	(empty)	0	0
1534723194.618889	CzrrYK2xngmJBQCHD	92.63.197.100	54355	3.110.29.207	22	-	-	-	S0	S	1	0	(empty)	0	0
1534723194.636590	C8BIea45er6At3ru1i	80.82.77.33	26366	3.110.70.55	

In [40]:
test(conn)

#separator \x09
#set_separator	,
#empty_field	(empty)
#unset_field	-
#path	anon.conn_tcp
#open	2018-08-19-20-00-00
#fields	ts	uid	id.orig_h	id.orig_p	id.resp_h	id.resp_p	duration	orig_bytes	resp_bytes	conn_state	history	orig_pkts	resp_pkts	tunnel_parents	local
#types	time	string	addr	port	addr	port	interval	count	count	string	string	count	count	set[string]	count
1534723192.365457	Co7jJo3ZWvcHNVJBp3	3.110.248.189	24472	13.107.5.88	443	2.234384	819	6093	RSTO	ShADadR	17	13	(empty)	1
1534723194.600236	ChNGWc2tYAUfXsSAOf	45.227.255.97	40351	3.110.130.108	8549	-	-	-	S0	S	1	0	(empty)	0
1534723194.603546	CcFOOY1Ws7QH2xI2Al	45.227.255.97	40340	3.110.95.246	5569	-	-	-	S0	S	1	0	(empty)	0
1534723194.604892	CZPyv12tvOYvGbdpMl	45.227.255.97	40340	3.110.3.29	5580	-	-	-	S0	S	1	0	(empty)	0
1534723194.607115	CK9nil4OJcbHqHXZk3	185.169.231.209	65535	3.110.152.124	8888	-	-	-	S0	S	1	0	(empty)	0
1534723194.607921	CAyQv12zpLXt4qOGLf	5.189.226.178	60000	3.110.122.8	7000	-	-	-	S0	S	1	0	(empty)	0
1534723194.612